# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [4]:
# Your code goes here
import pandas as pd

# Load three datasets 
df1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv')

#check columns similarities
columns_set_df1 = set(df1.columns)
columns_set_df2 = set(df2.columns)
columns_set_df3 = set(df3.columns)
print(columns_set_df1 )
print(columns_set_df2)
print(columns_set_df3)

#we saw they are problem  to match correctly 
def format_column_names(df):
    df.columns = df.columns.str.replace('ST', 'state')
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    return df

df1 = format_column_names(df1)
df2 = format_column_names(df2)
df3 = format_column_names(df3)

# check 
print(df1.columns)
print(df2.columns)
print(df3.columns)

# columns are now similar merging in approch but reordorning it first I think i can put it in the first function but i didn't think about before trying to merge 
df2 = df2[df1.columns]
df3 = df3[df1.columns]

# check 
print(df1.columns)
print(df2.columns)
print(df3.columns)




{'Vehicle Class', 'Income', 'ST', 'Number of Open Complaints', 'Customer', 'GENDER', 'Customer Lifetime Value', 'Monthly Premium Auto', 'Total Claim Amount', 'Education', 'Policy Type'}
{'Vehicle Class', 'Income', 'ST', 'Number of Open Complaints', 'Customer', 'GENDER', 'Customer Lifetime Value', 'Monthly Premium Auto', 'Total Claim Amount', 'Education', 'Policy Type'}
{'Vehicle Class', 'Income', 'Number of Open Complaints', 'Customer', 'Customer Lifetime Value', 'Monthly Premium Auto', 'State', 'Total Claim Amount', 'Education', 'Policy Type', 'Gender'}
Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')
Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'total_claim_amount', 'policy_type', 'vehicle_class'

In [6]:
#merge 

merged_df = pd.concat([df1, df2, df3], ignore_index=True)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   9137 non-null   object 
 1   state                      9137 non-null   object 
 2   gender                     9015 non-null   object 
 3   education                  9137 non-null   object 
 4   customer_lifetime_value    9130 non-null   object 
 5   income                     9137 non-null   float64
 6   monthly_premium_auto       9137 non-null   float64
 7   number_of_open_complaints  9137 non-null   object 
 8   policy_type                9137 non-null   object 
 9   vehicle_class              9137 non-null   object 
 10  total_claim_amount         9137 non-null   float64
dtypes: float64(3), object(8)
memory usage: 1.0+ MB
None


In [19]:


# cleaning 

merged_df['gender'] = merged_df['gender'].str.lower().replace({'f': 'F', 'femal': 'F', "female":"F",'m':"M","male": 'M'})
state_mapping = {'AZ': 'Arizona', 'Cali': 'California', 'WA': 'Washington'}
merged_df['state'] = merged_df['state'].replace(state_mapping)
merged_df['education'] = merged_df['education'].replace({'Bachelors': 'Bachelor'})
vehicle_class_mapping = {'Sports Car': 'Luxury', 'Luxury SUV': 'Luxury', 'Luxury Car': 'Luxury'}
merged_df['vehicle_class'] = merged_df['vehicle_class'].replace(vehicle_class_mapping)
merged_df['customer_lifetime_value'] = merged_df['customer_lifetime_value'].astype(str).str.replace('%', '').astype(float)

#cleaning  check
print(merged_df['gender'].unique())
print(merged_df['state'].unique())
print(merged_df['education'].unique())
print(merged_df['customer_lifetime_value'].unique())
print(merged_df['vehicle_class'].unique())

#numerical
merged_df['customer_lifetime_value'] = pd.to_numeric(merged_df['customer_lifetime_value'], errors='coerce')
is_numerical = pd.api.types.is_numeric_dtype(merged_df['customer_lifetime_value'])
is_numerical

# null 
cleaned_null = merged_df.dropna(axis=0, how='any').dropna(axis=1, how='any')
print(merged_df.info())

# duplicate
duplicates = merged_df.duplicated()
print(duplicates.sum())
df_first_occurrence = merged_df.drop_duplicates()
#check
no_duplicates_check = df_first_occurrence.duplicated().sum()
print("\nNumber of Duplicate Rows after Handling:", no_duplicates_check)

# Save the cleaned dataset to a new CSV file (index = false reboot index )
df_first_occurrence.to_csv('cleaned_dataset_first_occurrence.csv', index=False)

# look if it works
print(df_first_occurrence.info())


[nan 'F' 'M']
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon' nan]
['Master' 'Bachelor' 'High School or Below' 'College' 'Doctor' nan]
[           nan  697953.59     1288743.17     ...    8163.890428
    7524.442436    2611.836866]
['Four-Door Car' 'Two-Door Car' 'SUV' 'Luxury' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   9137 non-null   object 
 1   state                      9137 non-null   object 
 2   gender                     9015 non-null   object 
 3   education                  9137 non-null   object 
 4   customer_lifetime_value    9130 non-null   float64
 5   income                     9137 non-null   float64
 6   monthly_premium_auto       9137 non-null   float64
 7   number_of_open_complaints  9137 non-null   object 
 8   policy_type                9137 non

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [ ]:
# Your code goes here

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [25]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df4 = pd.read_csv(url)

#look
print(df4.columns)
print(df4.info())
duplicates = df4[df4.duplicated()]
num_duplicates = len(duplicates)
print(num_duplicates)


Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10910 non-null  object 
 3   customer_lifetime_value        1091

In [27]:
# All was good so go pivot 
pivot_table_result = pd.pivot_table(df4, values='total_claim_amount', index='sales_channel', aggfunc='sum')
pivot_table_result['total_claim_amount'] = round(pivot_table_result['total_claim_amount'], 2)

print(pivot_table_result)
#the not physic canal seems to makes less direct money 


               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [32]:
pivot_table_result = pd.pivot_table(df4, values='customer_lifetime_value', index=['gender', 'education'], aggfunc='mean')

# Round the average customer lifetime value to 2 decimal points
pivot_table_result['customer_lifetime_value'] = round(pivot_table_result['customer_lifetime_value'], 2)
print(pivot_table_result)




Pivot Table - Average Customer Lifetime Value per Gender and Education Level:
                             customer_lifetime_value
gender education                                    
F      Bachelor                              7874.27
       College                               7748.82
       Doctor                                7328.51
       High School or Below                  8675.22
       Master                                8157.05
M      Bachelor                              7703.60
       College                               8052.46
       Doctor                                7415.33
       High School or Below                  8149.69
       Master                                8168.83


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*